In [10]:
from python_dict_wrapper import wrap
import sys
sys.path.append('../')



In [44]:
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter

def get_anim(result, norm=False, title=""):
    fig = plt.figure(figsize=(4, 4))
    ax = plt.gca()
    im = plt.imshow([[1]], cmap='gray')
    
    if title:
        plt.suptitle(title)
    plt.axis('off')
    nt = result.shape[2]

    rg = None
    if norm:
        rg = [result.min(), result.max()]

    def drawframe(n):
        if n >= nt:
            n = nt - 1
        else:
            im.set_data(result[:, :, :, n])
        if norm:
            im.set_clim(rg)
        return (im)

    # blit=True re-draws only the parts that have changed.
    anim = animation.FuncAnimation(fig, drawframe, frames=nt*2, interval=500, blit=False)
    plt.close()
    return anim

for features in ['ShallowMonkeyNet_pvc1', 'ShallowMonkeyNet_pvc4', 'V1Net']:
    args = wrap({'features': features,
                 'ckpt_root': '../pretrained'})

    from fmri_models import get_feature_model

    model, _, _ = get_feature_model(args)

    if features == 'V1Net':
        w = model.s1.conv1.weight.cpu().detach().numpy()
    else:
        w = model.conv1.weight.cpu().detach().numpy()
    w = w / abs(w).max()
    ani = get_anim(.5 + .5*w.reshape((4, 4, 3, 5, 7, 7)).transpose((0, 4, 1, 5, 2, 3)).reshape((28, 28, 3, 5)), title=features)
        
    writer = FFMpegWriter(fps=10)
    ani.save(f"figures/filters_{features}.gif", writer=writer)  

In [15]:
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter

from pathlib import Path

import torch

def get_anim(result, norm=False, title=""):
    fig = plt.figure(figsize=(4, 4))
    ax = plt.gca()
    im = plt.imshow([[1]], cmap='gray')
    
    if title:
        plt.suptitle(title)
    plt.axis('off')
    nt = result.shape[2]

    rg = None
    if norm:
        rg = [result.min(), result.max()]

    def drawframe(n):
        if n >= nt:
            n = nt - 1
        else:
            im.set_data(result[:, :, :, n])
        if norm:
            im.set_clim(rg)
        return (im)

    # blit=True re-draws only the parts that have changed.
    anim = animation.FuncAnimation(fig, drawframe, frames=nt*2, interval=500, blit=False)
    plt.close()
    return anim

checkpoints = Path('../models/dorsalnet_slower/').glob('*.pt')

ws = []
for i, path in enumerate(sorted(list(checkpoints))[::10]):
    data = torch.load(str(path))
    w = data['subnet.s1.conv1.weight'].cpu().detach().numpy()
    w = w / abs(w).max()
    ani = get_anim(.5 + .5*w.reshape((4, 4, 3, 5, 7, 7)).transpose((0, 4, 1, 5, 2, 3)).reshape((28, 28, 3, 5)), title=f'Step {i}')
        
    writer = FFMpegWriter(fps=10)
    ani.save(f"figures/filters_dorsalnet_{i}.gif", writer=writer)

    ws.append(w.ravel())

In [18]:
import numpy as np

np.testing.assert_allclose(ws[0], ws[-1])

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 11759 / 11760 (100%)
Max absolute difference: 0.08925858
Max relative difference: 3036.3225
 x: array([ 0.051969, -0.028676, -0.064085, ...,  0.097489, -0.052117,
        0.026089], dtype=float32)
 y: array([ 0.063339, -0.005472, -0.032252, ...,  0.107188, -0.049879,
        0.032908], dtype=float32)